## Implementação de um Classificador Perceptron

In [1]:
import numpy as np


class Perceptron(object):
    """Perceptron classifier.

    Parameters
    ------------
    eta : float
      Learning rate (between 0.0 and 1.0)
    n_iter : int
      Passes over the training dataset.
    random_state : int
      Random number generator seed for random weight
      initialization.

    Attributes
    -----------
    w_ : 1d-array
      Weights after fitting.
    errors_ : list
      Number of misclassifications (updates) in each epoch.

    """
    def __init__(self, eta=0.01, n_iter=50, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state

    def fit(self, X, y):
        """Fit training data.

        Parameters
        ----------
        X : {array-like}, shape = [n_examples, n_features]
          Training vectors, where n_examples is the number of examples and
          n_features is the number of features.
        y : array-like, shape = [n_examples]
          Target values.

        Returns
        -------
        self : object

        """
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        self.errors_ = []
        
        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self

    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.net_input(X) >= 0.0, 1, -1)

## Testando o classificador Perceptron

In [2]:
"""Dados de Treinamento """
X = np.array([[1,1],[2,2],[3,3]])
y = np.array([1,1,-1])

"""Criando objeto Perceptron"""
ppn = Perceptron(eta=0.1, n_iter=10000)

"""Treinando o modelo"""
ppn.fit(X, y)

"""Testando modelo treinado """
X_newdata = np.array([[4,4],[2,2],[3,3]])
print("Resultado da Predição",ppn.predict(X_newdata));

Resultado da Predição [-1  1 -1]


## Questao 1 - Implemente uma função para calcular a acurácia do modelo

In [3]:
def accuracy(original, predicted):
    match = 0
    for i in range(len(original)):
        if original[i] == predicted[i]:
            match += 1
    return match / len(original) * 100.0


In [4]:
predicted = ppn.predict(X)
original = y
accuracy(original, predicted)

100.0

In [5]:
predicted = ppn.predict(X_newdata)
original = np.array([1,-1,-1])
accuracy(original, predicted)

33.33333333333333

## Questao 2 - Implemente um método de validação cruzada para testar 

In [63]:
import random
def split_folds(dataset, k):
    folds = list()
    fold_size = round(len(dataset) / k)
    aux = dataset.copy()
    np.random.shuffle(aux)
    i = j = 0
    for i in range(k):
        fold = list()
        while len(fold) < fold_size:
            if j >= len(dataset):
                break
            fold.append(aux[j])
            j+=1
        folds.append(fold)


    return folds

In [7]:
folds = split_folds(X, 2)
folds

[[array([3, 3]), array([1, 1])], [array([2, 2])]]

In [8]:
def cross_val(X, y, model, k):
    x_folds = np.array(split_folds(X, k))
    y_folds = np.array(split_folds(y, k))
    
    scores = list()
    for i in range(k):
        index = [j for j in range(k) if j!=i]
        X_train, y_train = np.concatenate(x_folds[index]), np.concatenate(y_folds[index])
        X_test, y_test = x_folds[i], y_folds[i] 
        model.fit(X_train, y_test)
        predicted = model.predict(X_test)
        scores.append(accuracy(y_test, predicted))
    print(scores)
    return sum(scores)/len(scores)
        

In [14]:
cross_val(X, y, ppn, 3)

[100.0, 100.0, 0.0]


66.66666666666667

## Teste o classificador usando um conjunto de dados linearmente separável e outro não linearmente separável
### Sugestão: crie datasets sintéticos com apenas dois atributos para voce poder visualizar a separação das classes

Linearmente separáveis - porta OR

In [15]:
x1 = np.array([[0,0],[0,1],[1,0],[1,1]])
y1 = np.array([0,1,1,1])
ppn.fit(x1,y1)
ppn.predict(x1)

array([1, 1, 1, 1])

In [16]:
accuracy(y1, ppn.predict(x1))

75.0

Não linearmente separável - porta XOR

In [17]:
x2 = np.array([[0,0],[0,1],[1,0],[1,1]])
y2 = np.array([0,1,1,0])
ppn.fit(x2,y2)
ppn.predict(x2)

array([1, 1, 1, 1])

In [18]:
accuracy(y2, ppn.predict(x2))

50.0

## Questao 3 - Treine um classificador perceptron para os dados de seu estudo de caso

### Importando dados


In [41]:
import sqlite3
import pandas as pd

In [42]:
db_sql = sqlite3.connect('database.sqlite')
df = pd.read_sql_query("SELECT * FROM BoardGames", db_sql)
db_sql.close()

### Pré-processando
O mesmo pré-processamento do trabalho 1 foi aplicado ao dataset, removendo colunas indesejadas e jogos irrelevantes. Também foram removidas as expansões de jogos

In [43]:
df = df[df['game.type']=='boardgame']
df = df.dropna(axis=0, subset=['details.yearpublished'])
df = df[df['stats.usersrated'] >= 10]
# Removendo colunas indesejadas
for col in df:
    if 'polls' in col or 'family' in col or 'subtype' in col:
        df.drop(col, axis=1, inplace=True)


df.drop('attributes.t.links.concat.2....', axis=1, inplace=True)
df.drop('details.description', axis=1, inplace=True)
df.drop('details.thumbnail', axis=1, inplace=True)
df.drop('details.image', axis=1, inplace=True)


In [44]:
usersrated_100 = df[df['stats.usersrated'] >= 100]
usersrated_100

,row_names,game.id,game.type,details.maxplayers,details.maxplaytime,details.minage,details.minplayers,details.minplaytime,details.name,details.playingtime,...,stats.bayesaverage,stats.median,stats.numcomments,stats.numweights,stats.owned,stats.stddev,stats.trading,stats.usersrated,stats.wanting,stats.wishing
0,1,1,boardgame,5.0,240.0,14.0,3.0,240.0,Die Macher,240.0,...,7.29168,0.0,1763.0,719.0,5251.0,1.59321,170.0,4498.0,505.0,1654.0
1,2,2,boardgame,4.0,30.0,12.0,3.0,30.0,Dragonmaster,30.0,...,5.87150,0.0,273.0,52.0,1053.0,1.46282,73.0,478.0,67.0,161.0
2,3,3,boardgame,4.0,60.0,10.0,2.0,30.0,Samurai,60.0,...,7.28295,0.0,3281.0,1355.0,11870.0,1.18531,234.0,12019.0,707.0,2601.0
3,4,4,boardgame,4.0,60.0,12.0,2.0,60.0,Tal der Könige,60.0,...,5.76636,0.0,111.0,30.0,523.0,1.21028,29.0,314.0,61.0,112.0
4,5,5,boardgame,6.0,90.0,12.0,3.0,90.0,Acquire,90.0,...,7.21895,0.0,5011.0,1515.0,18682.0,1.33020,823.0,15195.0,516.0,2219.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88630,88631,213492,boardgame,5.0,0.0,10.0,2.0,30.0,Pyramids,0.0,...,5.65671,0.0,25.0,3.0,221.0,1.15237,2.0,106.0,30.0,160.0
88729,88730,213893,boardgame,4.0,80.0,13.0,2.0,40.0,Yamataï,80.0,...,6.06010,0.0,110.0,24.0,728.0,1.45617,3.0,398.0,296.0,1832.0
89183,89184,216201,boardgame,6.0,120.0,12.0,2.0,20.0,Robo Rally (2016),120.0,...,5.98430,0.0,84.0,5.0,963.0,1.47625,8.0,341.0,99.0,469.0
89320,89321,216725,boardgame,5.0,60.0,14.0,1.0,30.0,Villages of Valeria: Deluxe Kickstarter Edition,60.0,...,5.73336,0.0,35.0,4.0,520.0,1.02655,16.0,119.0,7.0,5.0


### Selecionando colunas para usar no treinamento
As colunas selecionadas foram as relacionadas ao tempo de jogo e quantidade de jogadores

In [55]:
df_corr = ['stats.average', 'stats.averageweight']
for col in usersrated_100:
    if 'details' in col:
        df_corr.append(col)


In [58]:
df_ = usersrated_100[df_corr]
df_

,stats.average,stats.averageweight,details.maxplayers,details.maxplaytime,details.minage,details.minplayers,details.minplaytime,details.name,details.playingtime,details.yearpublished
0,7.66508,4.3477,5.0,240.0,14.0,3.0,240.0,Die Macher,240.0,1986.0
1,6.60815,1.9423,4.0,30.0,12.0,3.0,30.0,Dragonmaster,30.0,1981.0
2,7.44119,2.5085,4.0,60.0,10.0,2.0,30.0,Samurai,60.0,1998.0
3,6.60675,2.6667,4.0,60.0,12.0,2.0,60.0,Tal der Könige,60.0,1992.0
4,7.35830,2.5089,6.0,90.0,12.0,3.0,90.0,Acquire,90.0,1964.0
...,...,...,...,...,...,...,...,...,...,...
88630,7.02499,2.0000,5.0,0.0,10.0,2.0,30.0,Pyramids,0.0,2017.0
88729,7.48023,3.0417,4.0,80.0,13.0,2.0,40.0,Yamataï,80.0,2017.0
89183,7.45871,2.2000,6.0,120.0,12.0,2.0,20.0,Robo Rally (2016),120.0,2016.0
89320,7.52941,2.2500,5.0,60.0,14.0,1.0,30.0,Villages of Valeria: Deluxe Kickstarter Edition,60.0,2016.0


### Parametrização da dificuldade
Como o Perceptron é um modelo binário, a dificuldade do jogo foi dividida em duas categorias:
- Fácil (-1): para jogos com peso inferior a 3
- Difícil (1): para jogos com peso igual ou maior que 3

In [59]:
df_.loc[df_['stats.averageweight'] < 2.99, 'stats.averageweight'] = -1
df_.loc[df_['stats.averageweight'] >= 3, 'stats.averageweight'] = 1
df_


c:\users\romul\documents\ufc\s9\datascience\boardgamegeekanalysis\venv\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
c:\users\romul\documents\ufc\s9\datascience\boardgamegeekanalysis\venv\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,stats.average,stats.averageweight,details.maxplayers,details.maxplaytime,details.minage,details.minplayers,details.minplaytime,details.name,details.playingtime,details.yearpublished
0,7.66508,1.0,5.0,240.0,14.0,3.0,240.0,Die Macher,240.0,1986.0
1,6.60815,-1.0,4.0,30.0,12.0,3.0,30.0,Dragonmaster,30.0,1981.0
2,7.44119,1.0,4.0,60.0,10.0,2.0,30.0,Samurai,60.0,1998.0
3,6.60675,1.0,4.0,60.0,12.0,2.0,60.0,Tal der Könige,60.0,1992.0
4,7.35830,1.0,6.0,90.0,12.0,3.0,90.0,Acquire,90.0,1964.0
...,...,...,...,...,...,...,...,...,...,...
88630,7.02499,-1.0,5.0,0.0,10.0,2.0,30.0,Pyramids,0.0,2017.0
88729,7.48023,1.0,4.0,80.0,13.0,2.0,40.0,Yamataï,80.0,2017.0
89183,7.45871,-1.0,6.0,120.0,12.0,2.0,20.0,Robo Rally (2016),120.0,2016.0
89320,7.52941,-1.0,5.0,60.0,14.0,1.0,30.0,Villages of Valeria: Deluxe Kickstarter Edition,60.0,2016.0


### Criando o dataset de treinamento
As colunas usadas foram transformadas em vetores e agrupadas em um array Numpy.
A coluna dos pesos também foi transformada em array Numpy

In [99]:
dificuldade = df_['stats.averageweight']
max_players = df_['details.maxplayers']
min_players = df_['details.minplayers']
max_playtime = df_['details.maxplaytime']
min_playtime = df_['details.minplaytime']

X = np.array([max_players, min_players, max_playtime, min_playtime]).T
y = np.array(dificuldade)

In [102]:
X.shape, y.shape

((7465, 4), (7465,))

### Treinamento
O Perceptron foi criado utilizando 0.01 como taxa de aprendizagem e 1000 iterações. Os valores foram escolhidos de forma empírica por questão de performance e duração do treinamento.

In [103]:
model = Perceptron(0.01, 1000)
model.fit(X, y)

Treinamento concluído. O teste a seguir foi feito utilizando dados contidos no dataset de treino. O valor esperado de predição é 1, ou seja, um jogo difícil.

In [104]:
model.predict(np.array([5, 3, 240, 240]))

array(1)

O teste a seguir foi feito com dados não existentes no conjunto de treino. O valor esperado é -1, pois o peso original é cerca de 2.0.

In [106]:
model.predict(np.array([6.0, 20.0, 3.0, 20.0]))

array(-1)

### Acurácia e Cross Validation
O teste de validação cruzada foi feito para obter uma acurácia média do Perceptron. O k escolhido foi 10, devido a duração do treinamento.

In [105]:
cross_val(X, y, model, 10)

[71.17962466487936, 67.69436997319035, 69.16890080428955, 29.624664879356565, 66.89008042895442, 64.87935656836461, 68.09651474530831, 73.7265415549598, 72.11796246648794, 31.903485254691688]


61.52815013404826

A acurácia final do modelo foi de **61.5%**. Possívelmente seria possível aumentar essa acurácia utilizando um K maior e também variando os hiperparâmetros do modelo (taxa de aprendizagem e número de iterações). 

## Conclusão
O Perceptron simples, apesar de ter obtido resultados interessantes para o conjunto de dados apresentado, não é o modelo mais indicado para este dataset, visto que foi necessário alterar a classificação de dificuldade do jogo para adequar ao modelo binário. Um Perceptron multi-camadas, ou outros modelos multi-classe trariam informações mais relevantes para nós. Outra opção seria a utilização de um modelo de Regressão para tentar acertar a nota do jogo ao invés de sua dificuldade.